# Project 3

In this project we will train a NeRF model from scratch using our own data.

This notebook is from and uses [nerfstudio](https://docs.nerf.studio/). We encourage you to look through their documentation and try anything that looks interesting to you.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.



In [ ]:
#@markdown <h1>Install Colmap (~5 min)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge colmap
!colmap -h

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \

In [ ]:
#@markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
/content
Downloading...
From: https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t
To: /content/tinycudann-1.7-cp310-cp310-linux_x86_64.whl
100% 31.2M/31.2M [00:00<00:00, 103MB/s] 
Processing ./tinycudann-1.7-cp310-cp310-linux_x86_64.whl
tinycudann is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
/content
  Cloning https://github.com/nerfstudio-project/nerfstudio.git to /tmp/pip-req-build-syt6fp8f
  Running command git clone --filter=blob:none --quiet https://github.com/nerfstudio-project/nerfstudio.git /tmp/pip-req-build-syt6fp8f
  Resolved https://github.com/nerfstudio-project/nerfstudio.git to commit 64f0b2547ba02b1e2aeef719de95b4e7b1aa188a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata

In [4]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>1) Prior to this step, ensure you have prepared your own images/video. If you took a video, make sure to convert it to frames</h3>
#@markdown <h3>2) For scene, select upload your images. If you took a video and didn't convert to frames, be cautious of upload your own video. It can be very time consuming</h3>
#@markdown <h3>3) If you'd like, for testing purposes, you can use the existing dozer dataset.</h3>
#@markdown <h4> Note, data preperation can take some time. Once complete, you could zip and download the resulting data directory for later use if desired.
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDCE4 upload your images' #@param ['🚜 dozer',  '📤 upload your images' , '🎥 upload your own video']
scene = ' '.join(scene.split(' ')[1:])

if scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data/raw_images


Saving IMG_8551.jpeg to IMG_8551 (1).jpeg
Saving IMG_8552.jpeg to IMG_8552.jpeg
Saving IMG_8553.jpeg to IMG_8553.jpeg
Saving IMG_8554.jpeg to IMG_8554.jpeg
Saving IMG_8555.jpeg to IMG_8555.jpeg
Saving IMG_8556.jpeg to IMG_8556.jpeg
Saving IMG_8557.jpeg to IMG_8557.jpeg
Saving IMG_8558.jpeg to IMG_8558.jpeg
Saving IMG_8559.jpeg to IMG_8559.jpeg
Saving IMG_8560.jpeg to IMG_8560.jpeg
Saving IMG_8561.jpeg to IMG_8561.jpeg
Saving IMG_8562.jpeg to IMG_8562.jpeg
Saving IMG_8563.jpeg to IMG_8563.jpeg
Saving IMG_8564.jpeg to IMG_8564.jpeg
Saving IMG_8565.jpeg to IMG_8565.jpeg
Saving IMG_8566.jpeg to IMG_8566.jpeg
Saving IMG_8567.jpeg to IMG_8567.jpeg
Saving IMG_8568.jpeg to IMG_8568.jpeg
Saving IMG_8569.jpeg to IMG_8569.jpeg
Saving IMG_8570.jpeg to IMG_8570.jpeg
Saving IMG_8571.jpeg to IMG_8571.jpeg
Saving IMG_8572.jpeg to IMG_8572.jpeg
Saving IMG_8573.jpeg to IMG_8573.jpeg
Saving IMG_8574.jpeg to IMG_8574.jpeg
Saving IMG_8575.jpeg to IMG_8575.jpeg
Saving IMG_8576.jpeg to IMG_8576.jpeg
Saving I

In [5]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.821s
https://viewer.nerf.studio/?websocket_url=wss://major-dodos-buy.loca.lt
You may need to click Refresh Page after you start training!


In [6]:
#@markdown <h1>Start Training</h1>
#@markdown <h2>Here you can change any training parameters if desired such as the NeRF variant your training (sub instant-npg for a different model)</h2>

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train instant-ngp --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

Streaming output truncated to the last 5000 lines.
-------------------------------------------------------------------------------------------------------- 
26380 (87.93%)      56.600 ms            3 m, 24 s            33.69 K                                    
26390 (87.97%)      55.496 ms            3 m, 20 s            34.67 K                                    
26400 (88.00%)      61.688 ms            3 m, 42 s            33.82 K                                    
26410 (88.03%)      54.634 ms            3 m, 16 s            34.87 K                                    
26420 (88.07%)      54.143 ms            3 m, 13 s            35.23 K                                    
26430 (88.10%)      53.190 ms            3 m, 9 s             35.98 K                                    
26440 (88.13%)      52.413 ms            3 m, 6 s             36.57 K                                    
26450 (88.17%)      59.965 ms            3 m, 32 s            34.78 K                                

In [7]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/instant-ngp/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

/content/outputs/unnamed/instant-ngp/2023-12-07_192659


Saving camera_path.json to camera_path.json
/content
Loading latest checkpoint from load_dir
✅ Done loading checkpoint from outputs/unnamed/instant-ngp/2023-12-07_192659/nerfstudio_models/step-000029999.ckpt
Creating trajectory video
🎥 Rendering 🎥 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96/96(100.0%) 0.07 fps 0:00:00 0:22:35
╭──── 🎉 Render Complete 🎉 ─────╮
│         ╷                      │
│   Video │ renders/output.mp4   │
│         ╵                      │
╰────────────────────────────────╯
